In [ ]:
!pip install tenseal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 12.9 MB/s eta 0:00:00


In [ ]:
import tenseal as ts

# context=ts.context(
#         ts.SCHEME_TYPE.CKKS,
#         poly_modulus_degree=8192,
#         coeff_mod_bit_sizes=[60, 40, 40, 60]
#     )
# context.generate_galois_keys()
# context.global_scale=2 ** 40

context=ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[40, 21, 21, 21, 21, 21, 21, 40]
    )
context.generate_galois_keys()
context.global_scale=2 ** 21

In [ ]:
import numpy as np
import time
import psutil
import matplotlib.pyplot as plt
from scipy.special import expit
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from threading import Thread, Lock

In [ ]:
X_train = pd.read_csv('X_train3.csv')
X_train=X_train.to_numpy()
y_train = pd.read_csv('y_train3.csv')
y_train=y_train.to_numpy().reshape(-1)
X_test = pd.read_csv('X_test3.csv')
X_test=X_test.to_numpy()
y_test = pd.read_csv('y_test3.csv')
y_test=y_test.to_numpy().reshape(-1)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Lock for controlling CPU monitoring
monitor_lock = Lock()

In [ ]:
enc_X_train=[]
for i in X_train:
  p=ts.ckks_vector(context,i)
  enc_X_train.append(p)

In [ ]:
enc_y_train=[]
for i in y_train:
  p=ts.ckks_vector(context,[i])
  enc_y_train.append(p)

In [ ]:
def sigmoid(z):
  with monitor_lock:
    z=z.decrypt()
    z=round(z[0],7)
    x=1/(1+np.exp(-z))
    x=round(x,7)
    x=ts.ckks_vector(context,[x])
  return x

In [ ]:
def relinearlizing_vector(vec):
  vec_2=[]
  vec=vec.decrypt()
  for i in vec:
    p=round(i,7)
    vec_2.append(p)
  vec_2=ts.ckks_vector(context,vec_2)
  return vec_2

In [ ]:
def relinearlizing_scalar(sca):
  sca=sca.decrypt()
  sca=round(sca[0],7)
  sca=ts.ckks_vector(context,[sca])
  return sca

In [ ]:
n_samples,n_attributes=X_train.shape
n_samples,n_attributes

(1080, 22)

In [ ]:
weights=np.zeros(n_attributes)
bias=0
lr=0.01
weights,bias

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]),
 0)

In [ ]:
weights=ts.ckks_vector(context,weights)
bias=ts.ckks_vector(context,[bias])

In [ ]:
def get_cpu_and_ram_utilization(pid, duration):
    process = psutil.Process(pid)
    cpu_usages = []
    ram_usages = []
    start_time = time.time()
    while time.time() - start_time < duration:
        # with monitor_lock:
        cpu_usage = process.cpu_percent(interval=1)
        ram_usage = process.memory_percent()
        cpu_usages.append(cpu_usage)
        ram_usages.append(ram_usage)
        print(f"CPU utilization: {cpu_usage}% | RAM utilization: {ram_usage}%")
    total_cpu_usage = sum(cpu_usages)
    return total_cpu_usage, cpu_usages, ram_usages

In [ ]:
pid = os.getpid()

# Duration to monitor CPU usage
monitor_duration = 530

In [ ]:
time.sleep(15)

In [ ]:
results={}
def train_log_reg(enc_X_train,enc_y_train,weights,bias,lr):
  train_time_start=time.time()
  ar=np.arange(n_samples)
  for epoch in range(3):
    np.random.shuffle(ar)
    for i in ar:
      xi=enc_X_train[i]
      yi=enc_y_train[i]
      lin_op=xi.dot(weights)
      lin_op=lin_op+bias
      y_pred=sigmoid(lin_op)
      db=y_pred-yi
      dw=db*xi
      # dw=relinearlizing_vector(dw)
      # db=relinearlizing_scalar(db)
      # weights=relinearlizing_vector(weights)
      # bias=relinearlizing_scalar(bias)
      weights=weights-(lr*dw)
      bias=bias-(lr*db)
      # print(f'epoch={epoch}, {i} loop is executed')
  train_time_end=time.time()
  results['weights']=weights
  results['bias']=bias
  results['train_time']=train_time_end-train_time_start
  print(f'train time is {results["train_time"]}')

In [ ]:
train_thread = Thread(target=train_log_reg, args=(enc_X_train, enc_y_train, weights, bias, lr))
train_thread.start()

total_cpu_usage, cpu_usages, ram_usages = get_cpu_and_ram_utilization(pid, monitor_duration)
train_thread.join()

print(f"Total CPU utilization over {monitor_duration} seconds: {total_cpu_usage}%")

# Print the CPU and RAM usage per second
print("CPU usage per second:", cpu_usages)
print("RAM usage per second:", ram_usages)

# Calculate total CPU resource consumption in 'CPU-seconds'
cpu_seconds = sum(cpu_usages) / 100
print(f"Total CPU resource consumption: {cpu_seconds} CPU-seconds")

CPU utilization: 107.9% | RAM utilization: 19.187537526917804%
CPU utilization: 108.6% | RAM utilization: 19.187537526917804%
CPU utilization: 102.7% | RAM utilization: 19.187537526917804%
CPU utilization: 87.5% | RAM utilization: 19.189523914145937%
CPU utilization: 87.1% | RAM utilization: 19.189523914145937%
CPU utilization: 69.9% | RAM utilization: 19.189523914145937%
CPU utilization: 83.8% | RAM utilization: 19.189523914145937%
CPU utilization: 82.4% | RAM utilization: 19.189523914145937%
CPU utilization: 96.8% | RAM utilization: 19.189523914145937%
CPU utilization: 101.2% | RAM utilization: 19.189523914145937%
CPU utilization: 111.5% | RAM utilization: 19.189523914145937%
CPU utilization: 111.1% | RAM utilization: 19.189523914145937%
CPU utilization: 99.8% | RAM utilization: 19.189523914145937%
CPU utilization: 102.1% | RAM utilization: 19.189523914145937%
CPU utilization: 111.8% | RAM utilization: 19.189523914145937%
CPU utilization: 105.6% | RAM utilization: 19.189523914145937%

In [ ]:
weights_unenc=results['weights'].decrypt()
print(weights_unenc)
bias_unenc=results['bias'].decrypt()
print(bias_unenc)

In [ ]:
enc_X_test=[]
for i in X_test:
  p=ts.ckks_vector(context,i)
  enc_X_test.append(p)

In [ ]:
weights=results['weights']
bias=results['bias']

y_pred_unenc=[]
y_pred_enc=[]

test_time_start=time.time()
for i in enc_X_test:
  lin_op=i.dot(weights)
  lin_op=lin_op+bias
  y_pred=sigmoid(lin_op)
  y_pred_enc.append(y_pred)



for i in y_pred_enc:
  p=i.decrypt()[0]
  if p>=0.5:
    y_pred_unenc.append(1)
  else:
    y_pred_unenc.append(0)

test_time_end=time.time()
print(f'test time is {test_time_end-test_time_start}')

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_unenc)
print(f'Accuracy: {accuracy}')

In [ ]:
testing_time=test_time_end-test_time_start
print(testing_time)

In [ ]:
import joblib
joblib.dump((weights_unenc, bias_unenc, accuracy, y_pred_unenc, cpu_seconds, cpu_usages, ram_usages, results['train_time'], testing_time), 'variables_ckks2.pkl')